# Ejemplo de `word2vec` con `gensim`


En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [ ]:
import gensim, logging, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Entrenamiento de un modelo

Implemento una clase `Corpus` con un iterador sobre un directorio que contiene ficheros de texto. Utilizaré una instancia de `Corpus` para poder procesar de manera más eficiente una colección, sin necesidad de cargarlo previamente en memoria.

In [ ]:
class Corpus(object):
    '''Clase Corpus que permite leer de manera secuencial un directorio de documentos de texto'''
    
    def __init__(self, directorio):
        self.directory = directorio

    def __iter__(self):
        for fichero in os.listdir(self.directory):
            for linea in open(os.path.join(self.directory, fichero)):
                yield linea.split()


`CORPUSDIR` contiene una colección de noticias en español (normalizada previamente a minúsculas y sin signos de puntuación) con alrededor de 150 millones de palabras. Entrenamos un modelo en un solo paso, ignorando aquellos tokens que aparecen menos de 10 veces, para descartar erratas. 

In [ ]:
CORPUSDIR = 'PATH_TO_YOUR_CORPUS_DIRECTORY'
oraciones = Corpus(CORPUSDIR)
model = gensim.models.Word2Vec(oraciones, min_count=10, size=150, workers=2)

# el modelo puede entrenarse en dos pasos sucesivos pero por separado
#model = gensim.models.Word2Vec() # modelo vacío
#model.build_vocab(oraciones)  # primera pasada para crear la lista de vocabulario
#model.train(other_sentences)  # segunda pasada para calcula vectores

Una vez completado el entrenamiento (después de casi 30 minutos), guardamos el modelo en disco. 

In [ ]:
model.save('PATH_TO_YOUR_MODEL.w2v')

En el futuro, podremos utilizar este modelo cargándolo en memoria con la instrucción:

In [10]:
#model = gensim.models.Word2Vec.load('PATH_TO_YOUR_MODEL.w2v')
#model = gensim.models.Word2Vec.load('/data/w2v/eswiki-280.w2v')
model = gensim.models.Word2Vec.load('/data/w2v/efe.model.w2v')


2017-07-15 13:16:21,646 : INFO : loading Word2Vec object from /data/w2v/efe.model.w2v
2017-07-15 13:16:24,352 : INFO : loading syn1 from /data/w2v/efe.model.w2v.syn1.npy with mmap=None
2017-07-15 13:16:24,485 : INFO : loading syn0 from /data/w2v/efe.model.w2v.syn0.npy with mmap=None
2017-07-15 13:16:24,603 : INFO : setting ignored attribute syn0norm to None
2017-07-15 13:16:24,607 : INFO : setting ignored attribute cum_table to None
2017-07-15 13:16:24,609 : INFO : loaded /data/w2v/efe.model.w2v


## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 26 millones de términos:

In [11]:
print(model.corpus_count)

26522244


Cada término del vocabulario está representado como un vector con 150 dimensiones: 105 características. Podemos acceder al vector de un término concreto:

In [4]:
print(model['azul'], '\n')

print(model['verde'], '\n')

print(model['microsoft'])

[  4.96991217e-01   8.55721772e-01   6.82879806e-01  -4.14421827e-01
   1.72151077e+00  -8.15706775e-02  -3.84779096e-01   2.39795947e+00
  -1.08219063e+00   2.05436572e-01  -1.08851862e+00  -1.52230406e+00
  -3.02234702e-02   1.71446276e+00   9.03053999e-01   5.00934906e-02
  -2.77492136e-01   6.52989566e-01   2.99589902e-01  -1.67158830e+00
   7.22501874e-01   1.72892824e-01   2.41062713e+00  -1.96764266e+00
  -2.54679441e+00   3.38947445e-01   3.77230197e-01  -2.00717807e+00
  -1.37975141e-01   1.50688481e+00   9.42857385e-01   1.03348625e+00
  -3.24400783e+00   3.71784419e-01  -9.88212645e-01  -1.13850737e+00
   1.25355625e+00  -1.97994685e+00  -1.87854564e+00   1.29538226e+00
  -7.06246436e-01   1.55038989e+00   3.01234293e+00   7.06765711e-01
  -4.44420457e-01  -1.43686986e+00  -2.71735215e+00   1.69624233e+00
   1.35413718e+00   6.97062194e-01   1.51717079e+00   1.42068231e+00
   4.78212088e-01   2.25567269e+00   6.34011805e-01   1.51526082e+00
  -5.22151649e-01  -1.78797102e+00

Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [12]:
print('hombre - mujer', model.similarity('hombre', 'mujer'))

print('madrid - parís', model.similarity('madrid', 'parís'))

print('perro - gato', model.similarity('perro', 'gato'))

print('gato - periódico', model.similarity('gato', 'periódico'))

hombre - mujer 0.473325617835
madrid - parís 0.649728730699
perro - gato 0.631910988958
gato - periódico 0.180082365746


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [13]:
lista1 = 'madrid barcelona gonzález washington'.split()
print('en la lista', ' '.join(lista1), 'sobra:', model.doesnt_match(lista1))

lista2 = 'psoe pp ciu epi'.split()
print('en la lista', ' '.join(lista2), 'sobra:', model.doesnt_match(lista2))

lista3 = 'publicaron declararon soy negaron'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.doesnt_match(lista3))

lista3 = 'homero saturno cervantes shakespeare cela'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.doesnt_match(lista3))

2017-07-15 13:17:38,255 : INFO : precomputing L2-norms of word weight vectors


en la lista madrid barcelona gonzález washington sobra: gonzález
en la lista psoe pp ciu epi sobra: epi
en la lista publicaron declararon soy negaron sobra: soy
en la lista homero saturno cervantes shakespeare cela sobra: saturno


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [16]:
terminos = 'psoe chicago sevilla aznar podemos estuvieron'.split()
terminos = 'microsoft ibm iberia repsol'.split()

for t in terminos:
    print(t, '==>', model.most_similar(t), '\n')

microsoft ==> [('apple', 0.7117506861686707), ('toshiba', 0.7028107047080994), ('sony', 0.6914273500442505), ('chevron', 0.678649365901947), ('mci', 0.6718457937240601), ('corp', 0.6699045896530151), ('att', 0.667290210723877), ('mccaw', 0.6616576313972473), ('westinghouse', 0.6602280139923096), ('communications', 0.6471770405769348)] 

ibm ==> [('ecd', 0.7653704881668091), ('inf', 0.7394723892211914), ('ixc', 0.6570380330085754), ('ibs', 0.6323965787887573), ('ief', 0.6096996665000916), ('tfb', 0.597407877445221), ('nis', 0.55487060546875), ('icx', 0.5465259552001953), ('bnm', 0.5369136929512024), ('wsh', 0.5276572704315186)] 

iberia ==> [('aviaco', 0.8360572457313538), ('seat', 0.6662347316741943), ('sepla', 0.6475242376327515), ('spanair', 0.6450983285903931), ('gte', 0.6450628638267517), ('binter', 0.6434944868087769), ('renfe', 0.6096404790878296), ('viasa', 0.6077329516410828), ('pluna', 0.6002950072288513), ('trasmediterránea', 0.5986974835395813)] 

repsol ==> [('endesa', 0.77

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [15]:
print('==> alcalde + mujer - hombre')
most_similar = model.most_similar(positive=['alcalde', 'mujer'], negative=['hombre'], topn=3)
for item in most_similar:
    print(item)

print('==> madrid + filipinas - españa')
most_similar = model.most_similar(positive=['madrid', 'filipinas'], negative=['españa'], topn=3)
for item in most_similar:
    print(item)

    
    

==> alcalde + mujer - hombre
('alcaldesa', 0.5925700068473816)
('consellera', 0.5343092679977417)
('concejala', 0.5338941812515259)
==> madrid + filipinas - españa
('manila', 0.5159924030303955)
('pekín', 0.5065802335739136)
('hanoi', 0.49402812123298645)
